In [2]:
# imports
import pandas as pd
import numpy as np

# SQL
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, inspect
from sqlalchemy.sql import text

In [3]:
# create filepath
filepath = "Resources/listings_2024.csv"

# read the CSV file
df_2024 = pd.read_csv(filepath, encoding='latin1', low_memory=False)

#view
df_2024.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,1189243425411300671,The Grand III,61391963,Stay With Vibe,Manhattan,Murray Hill,40.750251,-73.977745,Entire home/apt,58.0,30,1,2024-08-02,0.86,130,357,1,NaN
1,651593916026998398,Cheerful one bedroom townhouse with backyard,136812643,Marah,Brooklyn,Flatlands,40.624760,-73.935560,Private room,80.0,30,0,NaN,NaN,1,365,0,NaN
2,310325,Large Sunny Bedroom with Bay Window,745069,Kimberly,Manhattan,Harlem,40.823590,-73.946010,Private room,75.0,30,31,2019-09-28,0.21,2,335,0,NaN
3,572612125615500056,Room by Sunny & Bay! Sunset Park & Bay Ridge,358089614,Joshua,Brooklyn,Sunset Park,40.639520,-74.014840,Private room,45.0,30,6,2024-07-30,0.21,2,336,1,NaN
4,1020282701018874374,Trendy Private Bedroom,483056418,Kristina,Brooklyn,Bedford-Stuyvesant,40.678946,-73.943759,Private room,47.0,30,0,NaN,NaN,24,269,0,NaN
5,640443472855597440,Lovely 2 bed/2 bath home with private balcony,24278208,Reah,Brooklyn,Bedford-Stuyvesant,40.687400,-73.954970,Private room,194.0,30,0,NaN,NaN,2,323,0,NaN
6,725482656106188093,Gorgeous ! Amazing! Rasta palace â¦.,4941291,Randon,Brooklyn,Bedford-Stuyvesant,40.680430,-73.957150,Entire home/apt,400.0,30,0,NaN,NaN,1,364,0,NaN
7,681730243323738679,Lovely one bedroom in Brooklyn,45774302,Julian,Brooklyn,Bushwick,40.694410,-73.922970,Private room,79.0,30,14,2023-01-01,0.56,2,89,0,NaN
8,1004979479809147241,Beautiful Room in Charming Apt,540041298,Kristina,Queens,Ridgewood,40.702610,-73.904622,Private room,45.0,30,0,NaN,NaN,5,260,0,NaN
9,969035784970111373,StuyvesantHieghts,534575586,Walter,Brooklyn,Bedford-Stuyvesant,40.683602,-73.930338,Entire home/apt,225.0,30,5,2023-10-16,0.42,1,180,5,NaN


In [4]:
# drop columns
df_2024.drop(columns=['name', 'host_id', 'host_name', 'last_review', 'license', 'number_of_reviews_ltm', 'latitude', 'longitude', 'calculated_host_listings_count'], inplace=True)

In [5]:
# check the drop
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37541 entries, 0 to 37540
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   37541 non-null  int64  
 1   neighbourhood_group  37541 non-null  object 
 2   neighbourhood        37541 non-null  object 
 3   room_type            37541 non-null  object 
 4   price                22790 non-null  float64
 5   minimum_nights       37541 non-null  int64  
 6   number_of_reviews    37541 non-null  int64  
 7   reviews_per_month    26001 non-null  float64
 8   availability_365     37541 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 2.6+ MB


In [6]:
# adjust data for some of the null values to still be useful
# if no minimum_nights, change the value to 1
df_2024['minimum_nights'] = df_2024['minimum_nights'].fillna(1)

# if no neighbourhood_group or neighbourhood, change to 'unknown' because the lat and lon will show us later
df_2024['neighbourhood_group'] = df_2024['neighbourhood_group'].fillna('unknown')
df_2024['neighbourhood'] = df_2024['neighbourhood'].fillna('unknown')

# if number of reviews has no information, change the value to 0
df_2024['number_of_reviews'] = df_2024['number_of_reviews'].fillna('0')

df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37541 entries, 0 to 37540
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   37541 non-null  int64  
 1   neighbourhood_group  37541 non-null  object 
 2   neighbourhood        37541 non-null  object 
 3   room_type            37541 non-null  object 
 4   price                22790 non-null  float64
 5   minimum_nights       37541 non-null  int64  
 6   number_of_reviews    37541 non-null  int64  
 7   reviews_per_month    26001 non-null  float64
 8   availability_365     37541 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 2.6+ MB


In [7]:
# take out the availability 365 column for now...
df_2024.drop(columns=['availability_365', 'reviews_per_month'], inplace=True)
df_2024.head(10)

,id,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews
0,1189243425411300671,Manhattan,Murray Hill,Entire home/apt,58.0,30,1
1,651593916026998398,Brooklyn,Flatlands,Private room,80.0,30,0
2,310325,Manhattan,Harlem,Private room,75.0,30,31
3,572612125615500056,Brooklyn,Sunset Park,Private room,45.0,30,6
4,1020282701018874374,Brooklyn,Bedford-Stuyvesant,Private room,47.0,30,0
5,640443472855597440,Brooklyn,Bedford-Stuyvesant,Private room,194.0,30,0
6,725482656106188093,Brooklyn,Bedford-Stuyvesant,Entire home/apt,400.0,30,0
7,681730243323738679,Brooklyn,Bushwick,Private room,79.0,30,14
8,1004979479809147241,Queens,Ridgewood,Private room,45.0,30,0
9,969035784970111373,Brooklyn,Bedford-Stuyvesant,Entire home/apt,225.0,30,5


In [8]:
# finally, drop the remaining null values that do not include lat/lon or the price
df_2024 = df_2024.dropna()
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22790 entries, 0 to 37538
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   22790 non-null  int64  
 1   neighbourhood_group  22790 non-null  object 
 2   neighbourhood        22790 non-null  object 
 3   room_type            22790 non-null  object 
 4   price                22790 non-null  float64
 5   minimum_nights       22790 non-null  int64  
 6   number_of_reviews    22790 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.4+ MB


In [9]:
# Double check the listings are spelled correctly
df_2024.neighbourhood_group.unique()

array(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

In [10]:
# save the final df to a new csv for use
df_2024.to_csv("Resources/2024_clean_data_only.csv")